In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [5]:
!kaggle competitions download -c bluebook-for-bulldozers

 97%|████████████████████████████████████▊ | 47.0M/48.4M [00:01<00:00, 52.2MB/s]
100%|██████████████████████████████████████| 48.4M/48.4M [00:01<00:00, 34.5MB/s]


In [10]:
!ls

KAGGLE_Bulldozer.ipynb data
